# Spinful fermions

In [1]:
from vmc_torch.hamiltonian_torch import spinful_Fermi_Hubbard_square_lattice_torch
from vmc_torch.experiment.tn_model import fTNModel
from vmc_torch.fermion_utils import fPEPS
import symmray as sr
import torch
import quimb.tensor as qtn


Lx = 2
Ly = 2
N_f = int(Lx*Ly)
n_fermions_per_spin = (Lx*Ly//2, Lx*Ly//2)
dtype = torch.float64

H = spinful_Fermi_Hubbard_square_lattice_torch(
    Lx, Ly, t=1.0, U=8.0, N_f=N_f, n_fermions_per_spin=None, no_u1_symmetry=True)
H_matrix = torch.tensor(H.to_dense(), dtype=dtype)

D = 4
flat = False
seed = 42
peps = sr.networks.PEPS_fermionic_rand(
    "Z2",
    Lx,
    Ly,
    D,
    phys_dim=[
        (0, 0),  # linear index 0 -> charge 0, offset 0
        (1, 1),  # linear index 1 -> charge 1, offset 1
        (1, 0),  # linear index 2 -> charge 1, offset 0
        (0, 1),  # linear index 3 -> charge 0, offset 1
    ],
    subsizes="equal",
    flat=flat,
    seed=seed,
)
peps_np = peps.copy()
peps.apply_to_arrays(lambda x: torch.tensor(x, dtype=dtype))
peps = fPEPS(peps) # register as fPEPS object, which has get_amp function
fpeps_model = fTNModel(peps, max_bond=-1, dtype=dtype)
all_states = torch.tensor(H.hilbert.all_states())
psi_vec = fpeps_model(all_states)
E_vmc = torch.vdot(psi_vec, H_matrix @ psi_vec) / torch.vdot(psi_vec, psi_vec)
print("Variational energy:", E_vmc.item())



H_terms = sr.ham_fermi_hubbard_from_edges(
    symmetry='Z2',
    edges=tuple(peps.gen_bond_coos()),
    t=1.0,
    U=8.0,
    mu=0.0,
)
ham = qtn.LocalHam2D(Lx, Ly, H_terms)
E_double_layer = peps_np.compute_local_expectation_exact(ham.terms, normalized=True)
print("Double layer contraction energy:", E_double_layer)

/home/sijingdu/TNVMC/VMC_code/clean_symmray/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
F=2.16 C=3.35 S=4.00 P=6.32: 100%|██████████| 10/10 [00:00<00:00, 61.45it/s]


Variational energy: 9.850763601060661
Double layer contraction energy: 9.850763601060663


# Spinless fermions

In [ ]:
from vmc_torch.hamiltonian_torch import spinless_Fermi_Hubbard_square_lattice_torch
from vmc_torch.experiment.tn_model import fTNModel
from vmc_torch.fermion_utils import fPEPS
import symmray as sr
import torch
import quimb.tensor as qtn


Lx = 2
Ly = 2
N_f = None
dtype = torch.float64

H = spinless_Fermi_Hubbard_square_lattice_torch(
    Lx, Ly, t=1.0, V=8.0, N_f=N_f, pbc=False)
H_matrix = torch.tensor(H.to_dense(), dtype=dtype)

D = 4
flat = False
seed = 42
peps = sr.networks.PEPS_fermionic_rand(
    "Z2",
    Lx,
    Ly,
    D,
    phys_dim=2,
    subsizes="equal",
    flat=flat,
    seed=seed,
)
peps_np = peps.copy()
peps.apply_to_arrays(lambda x: torch.tensor(x, dtype=dtype))
peps = fPEPS(peps) # register as fPEPS object, which has get_amp function
fpeps_model = fTNModel(peps, max_bond=-1, dtype=dtype)
all_states = torch.tensor(H.hilbert.all_states())
psi_vec = fpeps_model(all_states)
E_vmc = torch.vdot(psi_vec, H_matrix @ psi_vec) / torch.vdot(psi_vec, psi_vec)
print("Variational energy:", E_vmc.item())


H_terms = sr.ham_fermi_hubbard_spinless_from_edges(
    symmetry='Z2',
    edges=tuple(peps.gen_bond_coos()),
    t=1.0,
    V=8.0,
)
ham = qtn.LocalHam2D(Lx, Ly, H_terms)
E_double_layer = peps_np.compute_local_expectation_exact(ham.terms, normalized=True)
print("Double layer contraction energy:", E_double_layer)